In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, LeakyReLU
from tensorflow.keras.optimizers import Adam

# Load and preprocess MNIST
(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5  # Scale to [-1, 1]
X_train = np.expand_dims(X_train, axis=-1)  # (N, 28, 28, 1)

latent_dim = 100

# Generator
def build_generator():
    model = Sequential([
        Dense(7*7*256, use_bias=False, input_shape=(latent_dim,)),
        BatchNormalization(),
        LeakyReLU(),
        Reshape((7, 7, 256)),
        Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),
        Conv2DTranspose(64, (5,5), strides=(2,2), padding='same', use_bias=False),
        BatchNormalization(),
        LeakyReLU(),
        Conv2DTranspose(1, (5,5), strides=(2,2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator
def build_discriminator():
    model = Sequential([
        Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=[28,28,1]),
        LeakyReLU(0.2),
        Dropout(0.3),
        Conv2D(128, (5,5), strides=(2,2), padding='same'),
        LeakyReLU(0.2),
        Dropout(0.3),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    return model

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()

# Print architectures
print("Generator architecture:")
generator.summary()
print("\nDiscriminator architecture:")
discriminator.summary()

# Compile models
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
discriminator.trainable = False

from tensorflow.keras import Input, Model
z = Input(shape=(latent_dim,))
img = generator(z)
valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training
epochs = 10000
batch_size = 128
sample_interval = 2000

d_losses, g_losses = [], []

for epoch in range(1, epochs + 1):
    # Train Discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_imgs = X_train[idx]
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_imgs = generator.predict(noise)
    
    d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((batch_size, 1)))
    d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((batch_size, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train Generator
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))
    
    # Store losses
    d_losses.append(d_loss[0])
    g_losses.append(g_loss)
    
    # Visualize progress
    if epoch % sample_interval == 0 or epoch == 1:
        print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r*c, latent_dim))
        gen_imgs = generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5  # Scale to [0, 1]
        fig, axs = plt.subplots(r, c)
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[i*c + j].reshape(28, 28), cmap='gray')
                axs[i, j].axis('off')
        plt.suptitle(f'Generated Images at Epoch {epoch}')
        plt.show()

# Plot loss curves
plt.figure(figsize=(10,5))
plt.plot(d_losses, label="Discriminator loss")
plt.plot(g_losses, label="Generator loss")
plt.title("DCGAN Losses during Training")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()
